In [6]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

api_key1 = os.getenv("PINECONE_API_KEY")


pc = Pinecone(api_key=api_key1)
index = pc.Index("medibot")


In [7]:
embeddings = HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-L6-v2')

In [9]:
from langchain_pinecone import PineconeVectorStore
index_name = 'medibot'
vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [10]:
retriever = vector_store.as_retriever(search_type='similarity', search_kawgs={'k' : 4})

In [11]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template = '''You are a helpful assistant. Respond to the user queries only using the following context provided,
    context: {context}, query: {query}
    If there is not specific answer then respond I dont know.
    ''',
    input_variables=['context','query']
)

query = 'What causes cardiac arrest?'
relevant_docs = retriever.invoke(query)

In [12]:
def format_docs(retrieved_docs):
  context = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context

In [13]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [14]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'query' : RunnablePassthrough()
})

In [18]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id='openai/gpt-oss-20b',
    task='text-generation',
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_ACCESS_TOKEN")
)

model = ChatHuggingFace(llm=llm)

In [19]:
parser = StrOutputParser()

main_chain = parallel_chain | prompt | model | parser

In [20]:
main_chain.invoke('What causes cardiac arrest?')

'Cardiac arrest is caused by a sudden, often fatal disruption in the heart’s rhythm—most commonly ventricular fibrillation or ventricular tachycardia. These arrhythmias usually arise from underlying heart disease (such as a heart attack, myocarditis, or other structural problems) that interfere with the heart’s ability to pump effectively. When the heart stops beating or the rhythm becomes dangerously irregular, oxygen‑filled blood no longer reaches the brain and other vital organs, creating an emergency.'